In [1]:
# Athenticate Google Drive where the data is stored
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
# Install GPU version 
!pip install paddlepaddle-gpu

     |████████████████████████████████| 236.3MB 150kB/s 
     |████████████████████████████████| 901kB 36.4MB/s 
     |████████████████████████████████| 12.8MB 28.7MB/s 
     |████████████████████████████████| 122kB 41.0MB/s 
  Created wheel for rarfile: filename=rarfile-3.1-cp36-none-any.whl size=24908 sha256=110a10f6bc5af52ac585032715189db9c448c312ab530e815a0ee690c20e89c4
  Stored in directory: /root/.cache/pip/wheels/23/3c/c8/0215b6a5079492eff3be3f545ae0b0c4a66734c35c9e444eac
Successfully built rarfile
ERROR: plotnine 0.5.1 has requirement matplotlib>=3.0.0, but you'll have matplotlib 2.2.4 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: paddlepaddle-gpu 1.5.2.post107 has requirement scipy<=1.2.1,>=0.19.0, but you'll have scipy 1.3.1 which is incompatible.
  Found existing installation: matplotlib 3.0.3
    Uninstalling matplotlib-3.0.3:
      Successfully uninstalled matplotlib-3.0.

In [3]:
# Get the ERNIE pretrained model (base version)
!wget https://ernie.bj.bcebos.com/ERNIE_Base_en_stable-2.0.0.tar.gz

--2019-10-22 13:34:34--  https://ernie.bj.bcebos.com/ERNIE_Base_en_stable-2.0.0.tar.gz
Resolving ernie.bj.bcebos.com (ernie.bj.bcebos.com)... 103.235.46.61
Connecting to ernie.bj.bcebos.com (ernie.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405413980 (387M) [application/x-gzip]
Saving to: ‘ERNIE_Base_en_stable-2.0.0.tar.gz’

ERNIE_Base_en_stabl 100%[===================>] 386.63M  17.7MB/s    in 50s     

2019-10-22 13:35:25 (7.74 MB/s) - ‘ERNIE_Base_en_stable-2.0.0.tar.gz’ saved [405413980/405413980]



In [0]:
# Unzip 
!gunzip ERNIE_Base_en_stable-2.0.0.tar.gz

In [5]:
# Extract
!tar -xvf ERNIE_Base_en_stable-2.0.0.tar

ernie_config.json
params/
params/encoder_layer_4_post_att_layer_norm_scale
params/encoder_layer_2_multi_head_att_query_fc.b_0
params/encoder_layer_2_ffn_fc_1.w_0
params/encoder_layer_6_multi_head_att_key_fc.b_0
params/encoder_layer_7_multi_head_att_value_fc.w_0
params/encoder_layer_11_multi_head_att_value_fc.b_0
params/encoder_layer_5_multi_head_att_query_fc.w_0
params/encoder_layer_1_multi_head_att_value_fc.w_0
params/encoder_layer_11_post_ffn_layer_norm_scale
params/encoder_layer_7_multi_head_att_query_fc.w_0
params/encoder_layer_1_post_ffn_layer_norm_bias
params/encoder_layer_9_ffn_fc_1.b_0
params/encoder_layer_4_multi_head_att_query_fc.w_0
params/encoder_layer_2_ffn_fc_1.b_0
params/encoder_layer_7_post_ffn_layer_norm_bias
params/encoder_layer_7_post_att_layer_norm_scale
params/encoder_layer_6_multi_head_att_value_fc.w_0
params/encoder_layer_10_multi_head_att_output_fc.w_0
params/encoder_layer_5_multi_head_att_output_fc.w_0
params/encoder_layer_3_multi_head_att_output_fc.w_0
params/

In [6]:
# Clone the ERNIE repo down the to the vm attached to the notebook
!git clone https://github.com/PaddlePaddle/ERNIE.git

Cloning into 'ERNIE'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 1095 (delta 26), reused 67 (delta 18), pack-reused 1006
Receiving objects: 100% (1095/1095), 15.40 MiB | 23.54 MiB/s, done.
Resolving deltas: 100% (607/607), done.


In [0]:
# Get the data and fuse two columns of (my) data into a new one separated with a space
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks2/train.tsv', delimiter='\t')
df['tiab']  = df['ti'].astype(str) + " \n\n " + df['ab'].astype(str)
df = df[['tiab', 'y_stroke']]

In [8]:
# checking import
df.head()

,tiab,y_stroke
0,Effects of N-acetylcysteine on semen parameter...,0
1,Steady-state bioavailability of prescription o...,0
2,Favorable long-term results of primary pterygi...,0
3,Pilot study in humans on the pharmacokinetics ...,0
4,Neuroendocrine effects of a 20-mg citalopram i...,0


In [0]:
# Setting up label columns
import re
df.columns = ['text_a', 'label']

#df['label'] = df['label'].apply(lambda x: 1 if int(x) > 5 else 0) no need we do not have scores - we already have 1/0
df['text_a'] = df['text_a'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', str(x)))

In [10]:
# Checking again
df.head()

,text_a,label
0,Effects of N acetylcysteine on semen parameter...,0
1,Steady state bioavailability of prescription o...,0
2,Favorable long term results of primary pterygi...,0
3,Pilot study in humans on the pharmacokinetics ...,0
4,Neuroendocrine effects of a mg citalopram i...,0


In [11]:
# Train / Test splits (Note I use val = validation data instead their standard dev data description)
train = df.iloc[:, :int(len(df)*0.8)]
val = df.iloc[:, int(len(df)*0.8): ]

train.to_csv('train.tsv', index=False, sep='\t')
val.to_csv('val.tsv', index=False, sep='\t')

print("saved")

saved


In [0]:
# Getting the test data (remember to also put this into the same format - so I fuse the columns again into one)
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks2/test.tsv', sep='\t')
test['tiab']  = test['ti'].astype(str) + " \n\n " + test['ab'].astype(str)
test = test[['tiab']]

In [13]:
# Checking 
test.head()

,tiab
0,A 7-year prospective radiographic evaluation o...
1,Life review based on remembering specific posi...
2,Effects of chromium and chromium + vitamin C c...
3,One year treatment effects of the eruption gui...
4,The effects of acute aerobic exercise on the a...


In [14]:
test['tiab'] = test['tiab'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', str(x)))
test.to_csv('test.tsv', index=False, sep='\t')
print("saved test file")

saved test file


In [0]:
!mkdir -p dataset/SST-2

In [0]:
!mkdir -p parameters/params

In [0]:
!mv train.tsv dataset/SST-2/

In [0]:
!mv val.tsv dataset/SST-2/

In [0]:
!mv test.tsv dataset/SST-2/

In [0]:
!mv params/ parameters/params/

In [0]:
!mv dataset ERNIE/

In [0]:
!mv parameters ERNIE/

In [0]:
!cd ERNIE/

In [0]:
import os
os.environ['TASK_DATA_PATH'] = 'dataset'
os.environ['MODEL_PATH'] = 'parameters'

In [0]:
!sh script/en_glue/ernie_base/SST-2/task.sh

sh: 0: Can't open script/en_glue/ernie_base/SST-2/task.sh


In [25]:
os.chdir('ERNIE/')
os.environ['TASK_DATA_PATH']='dataset'
os.environ['MODEL_PATH']='parameters/params'
print(os.getcwd())


/content/ERNIE


In [0]:
!sh script/en_glue/ernie_base/SST-2/task.sh

script/en_glue/ernie_base/SST-2/task.sh: 7: script/en_glue/ernie_base/SST-2/task.sh: [[: not found
[INFO] 2019-10-08 11:16:59,647 [     args.py:   66]:	-----------  Configuration Arguments -----------
[INFO] 2019-10-08 11:16:59,647 [     args.py:   68]:	batch_size: 32
[INFO] 2019-10-08 11:16:59,647 [     args.py:   68]:	checkpoints: ./checkpoints
[INFO] 2019-10-08 11:16:59,647 [     args.py:   68]:	chunk_scheme: IOB
[INFO] 2019-10-08 11:16:59,648 [     args.py:   68]:	decr_every_n_nan_or_inf: 2
[INFO] 2019-10-08 11:16:59,648 [     args.py:   68]:	decr_ratio: 0.8
[INFO] 2019-10-08 11:16:59,648 [     args.py:   68]:	dev_set: dataset/SST-2/dev.tsv
[INFO] 2019-10-08 11:16:59,648 [     args.py:   68]:	diagnostic: None
[INFO] 2019-10-08 11:16:59,648 [     args.py:   68]:	diagnostic_save: None
[INFO] 2019-10-08 11:16:59,648 [     args.py:   68]:	do_lower_case: True
[INFO] 2019-10-08 11:16:59,648 [     args.py:   68]:	do_test: True
[INFO] 2019-10-08 11:16:59,648 [     args.py:   68]:	do_train:

In [0]:
# Evaluation Following soon here